In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from datasets import Dataset, DatasetDict, ClassLabel
from transformers import BertModel, ViTModel, BertTokenizer, ViTImageProcessor, Trainer, TrainingArguments, EvalPrediction
from PIL import Image, UnidentifiedImageError, ImageFile

from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score, precision_score
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from imblearn.over_sampling import RandomOverSampler

ImageFile.LOAD_TRUNCATED_IMAGES = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


DATASET_PATH = Path('data/MEC/')
MODEL_OUTPUT_DIR = Path('data/results')
DATASET_CACHE = Path('data/cache')
TEXT_COLUMN = 'text'
IMAGE_PATH_COLUMN = 'image_path'
TARGET_COLUMN = 'formal_register'
SEED = 42
N_EPOCHS = 30
BATCH_SIZE = 32


DATASET = DATASET_PATH / 'mec-dataset.csv'

def dataset_cache(stage: str):
    return str(DATASET_CACHE / stage)

Path(DATASET_CACHE).mkdir(parents=True, exist_ok=True)
Path(DATASET_CACHE).mkdir(parents=True, exist_ok=True)
print(sys.version)

3.11.9 (main, Apr 19 2024, 16:48:06) [GCC 11.2.0]


## Prepare Dataset

In [2]:
df = pd.read_csv(DATASET, index_col=0)
df[['cohesion', 'thematic_coherence', 'formal_register', 'text_typology']] = 'Nível ' + df[['cohesion', 'thematic_coherence', 'formal_register', 'text_typology']].astype(str)
df[IMAGE_PATH_COLUMN] =  df[IMAGE_PATH_COLUMN].apply(DATASET_PATH.joinpath).astype(str)
df = df.convert_dtypes()
df.info()
df.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 1188 entries, 0 to 1187
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   text                  1188 non-null   string
 1   motivating_situation  1188 non-null   string
 2   image_url             1188 non-null   string
 3   image_path            1188 non-null   string
 4   cohesion              1188 non-null   string
 5   thematic_coherence    1188 non-null   string
 6   formal_register       1188 non-null   string
 7   text_typology         1188 non-null   string
dtypes: string(8)
memory usage: 83.5 KB


,text,motivating_situation,image_url,image_path,cohesion,thematic_coherence,formal_register,text_typology
0,( O chorrinho nino ) - Eu est...,Eu encontrei em cima do armário alguns potes c...,https://storage.googleapis.com/ciclos-10698-bu...,data/MEC/MEC/Rc7dMxTP7ZdLNEvmF0jo/iet1QFw2ARNk...,Nível 3,Nível 3,Nível 3,Nível 4
1,.As meninas do potes de Tintas [T] Uma vez eu ...,Eu encontrei em cima do armário alguns potes c...,https://storage.googleapis.com/ciclos-10698-bu...,data/MEC/MEC/Rc7dMxTP7ZdLNEvmF0jo/F80gTOBoh2Lk...,Nível 3,Nível 3,Nível 3,Nível 4


In [3]:
class_names = sorted(df[TARGET_COLUMN].unique().tolist())

class_label = ClassLabel(num_classes=len(class_names), names=class_names)
class_label

ClassLabel(names=['Nível 1', 'Nível 2', 'Nível 3', 'Nível 4', 'Nível 5'], id=None)

## Training Functions

In [4]:
df[IMAGE_PATH_COLUMN].apply(Path).apply(Path.exists).all()

np.True_

In [5]:
class ClassfierHead(nn.Module):
    def __init__(self, input_size: int, hidden_size = 256, num_classes = 5):
        super().__init__()
        self.fn = nn.Linear(input_size, hidden_size)
        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, combined_last_hidden_state, labels=None):
        result = self.fn(combined_last_hidden_state)
        result = torch.relu_(result)
        logits = self.classifier(result)

        if labels is not None:
            loss = nn.functional.cross_entropy(logits, labels)
            return {"loss": loss, "logits": logits}
        
        return {'logits': logits}


BERT_MODEL_NAME = 'neuralmind/bert-base-portuguese-cased'
VIT_MODEL_NAME = 'google/vit-base-patch16-224'

bert_model: BertModel = BertModel.from_pretrained(BERT_MODEL_NAME).requires_grad_(False)
vit_model: ViTModel = ViTModel.from_pretrained(VIT_MODEL_NAME).requires_grad_(False)

tokenizer: BertTokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)
processor: ViTImageProcessor = ViTImageProcessor.from_pretrained(VIT_MODEL_NAME)

# Instanciar o cabeça de classificação
model = ClassfierHead(bert_model.config.hidden_size + vit_model.config.hidden_size)

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/gabriel/miniconda3/envs/project-hg/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
def is_image_valid(image_path):
    try:
        with Image.open(image_path) as img:
            img.verify()
        return True
    except (IOError, UnidentifiedImageError):
        return False

def preprocess_text_and_image(input_ids: torch.Tensor, attention_mask: torch.Tensor, pixel_values: torch.Tensor):
    # Processar o texto usando o BERT

    bert_model.eval().cuda()
    vit_model.eval().cuda()

    with torch.no_grad():
        bert_outputs = bert_model(input_ids=input_ids.cuda(), attention_mask=attention_mask.cuda())
        bert_pooled_output = bert_outputs.last_hidden_state[:, 0, :].detach().cpu()
    
        # Processar a imagem usando o ViT
        vit_outputs = vit_model(pixel_values=pixel_values.cuda())
        vit_pooled_output = vit_outputs.last_hidden_state[:, 0, :].detach().cpu()
    
        # Concatenar as saídas
        combined = torch.cat((bert_pooled_output, vit_pooled_output), dim=1)

    return {
        'combined_last_hidden_state': combined
    }


def prepare_dataset(
    df: pd.DataFrame,
    train_indexes: list[int],
    test_indexes: list[int],
    processor: ViTImageProcessor,
    tokenizer: BertTokenizer,
    text_column: str = 'text',
    image_path_column: str = 'image_path',
    target: str = 'formal_register',
    balanced: bool = True,
) -> DatasetDict:
    # Processing dataset e and storing in file cache
    COLUMNS = 'cohesion', 'thematic_coherence', 'formal_register', 'text_typology'
    original_dataset = (Dataset
        .from_pandas(df)
        .select_columns([text_column, image_path_column, *COLUMNS])
        .filter(is_image_valid, input_columns=[image_path_column])
        .map(lambda path: {'pixel_values': Image.open(path)}, input_columns=[image_path_column])
        .map(
            lambda pixel_values: processor(pixel_values, return_tensors='pt'),
            input_columns=['pixel_values'],
            batched=True,
            batch_size=8,
            num_proc=4,
            cache_file_name=dataset_cache('process-image.arrow'),
        )
        .map(
            lambda text: tokenizer(
                text,
                padding='max_length',
                truncation=True,
                max_length=512,
            ),
            batched=True,
            input_columns=[text_column],
            cache_file_name=dataset_cache('process-text.arrow'),
        )
        .with_format('pt')
        .map(
            preprocess_text_and_image,
            batched=True,
            batch_size=64,
            input_columns=['input_ids', 'attention_mask', 'pixel_values'],
            cache_file_name=dataset_cache('process-model-input.arrow'),
        )
        .remove_columns([image_path_column])
        .cast_column(target, class_label)
        .rename_column(target, 'labels')
    )

    # Balaced indexes
    if balanced:
        train_indexes = RandomOverSampler(random_state=SEED)\
            .fit_resample(train_indexes[None].T, df.loc[train_indexes, target])[0].flatten()

    # Creating dataset split
    dataset = DatasetDict(
        train=original_dataset.select(train_indexes),
        test=original_dataset.select(test_indexes)
    )
    return dataset


def compute_metrics(eval_preds: EvalPrediction, compute_result=False, *, PREDS: list = [], LABELS: list = []):
    labels = eval_preds.label_ids
    preds = eval_preds.predictions.argmax(-1)

    PREDS.append(preds)
    LABELS.append(labels)

    if compute_result:
        preds = torch.concat(PREDS).numpy(force=True)
        labels = torch.concat(LABELS).numpy(force=True)

        f1 = f1_score(labels, preds, average="weighted")
        acc = accuracy_score(labels, preds)

        PREDS, LABELS = [], []

        return {"accuracy": acc, "f1": f1}


def evaluate_model(model, dataset: DatasetDict, device):
    # Get model predictions and ground truth
    model.eval()
    model.to(device)
    torch.cuda.empty_cache()

    with torch.no_grad():
      batches = (batch['combined_last_hidden_state'] for batch in dataset['test'].iter(BATCH_SIZE))
      logits = [model(batch.to(device))['logits'] for batch in batches]
      logits = torch.concat(logits, 0)

    y_pred = torch.argmax(logits, dim=-1).cpu().numpy()
    y_true = dataset["test"]["labels"].cpu().numpy()

    torch.cuda.empty_cache()

    # Evaluate model
    return dict(
        report = classification_report(y_true, y_pred),
        accuracy = accuracy_score(y_true, y_pred),
        weighted_precision = precision_score(y_true, y_pred, average="weighted"),
        weighted_recall = recall_score(y_true, y_pred, average="weighted"),
        weighted_f1 = f1_score(y_true, y_pred, average="weighted"),
        macro_precision = precision_score(y_true, y_pred, average="macro"),
        macro_recall = recall_score(y_true, y_pred, average="macro"),
        macro_f1 = f1_score(y_true, y_pred, average="macro"),
    )

def mean_evaluation(results):
    from collections import defaultdict
    data = defaultdict(list)
    for result in results:
        for key, value in result.items():
            if key == 'report': continue
            data[key].append(value)
    return {
        key: np.mean(value)
        for key, value in data.items()
    }


In [7]:
# train_indexes, test_indexes = next(StratifiedShuffleSplit(random_state=SEED, test_size=0.2).split(df, df[TARGET_COLUMN]))
# dataset = prepare_dataset(df, train_indexes, test_indexes, processor, tokenizer)]
# dataset

In [8]:
def train_model(target: str, balanced: bool = True, test_size: float = 0.2):
    train_indexes, test_indexes = next(StratifiedShuffleSplit(random_state=SEED, test_size=test_size).split(df, df[TARGET_COLUMN]))

    dataset = prepare_dataset(df, train_indexes, test_indexes, processor, tokenizer, target=target, balanced=balanced)
    OUTPUT_DIR = f'results/training/{target}/'

    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=32,
        num_train_epochs=20,
        batch_eval_metrics=True,
        dataloader_num_workers=8,
        data_seed=SEED,
        eval_strategy="steps",
        eval_steps=100,
        save_steps=500,
        logging_dir=OUTPUT_DIR + "logging",
        logging_steps=25,
        fp16=True,
        learning_rate=2e-4,
        save_total_limit=2,
        use_cpu=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        compute_metrics=compute_metrics,
    )

    results = trainer.train()

    evaluation = evaluate_model(model, dataset, device)

    return dict(
        model = model,
        dataset = dataset,
        training = results,
        evaluation = evaluation,
    )


def cross_validate(target: str, balanced: bool = True):
    training_results = []
    evaluation_results = []

    for fold, (train_indexes, test_indexes) in enumerate(StratifiedKFold(random_state=SEED, n_splits=5, shuffle=True).split(df, df[TARGET_COLUMN]), start=1):
    
        dataset = prepare_dataset(df, train_indexes, test_indexes, processor, tokenizer, target=target, balanced=balanced)
        input_size = dataset['test'][0]['combined_last_hidden_state'].size()[0]

        model = ClassfierHead(input_size)

        OUTPUT_DIR = f'results/cross_validation/{target}/fold_{fold}/'

        training_args = TrainingArguments(
            output_dir=OUTPUT_DIR,
            per_device_train_batch_size=32,
            num_train_epochs=20,
            batch_eval_metrics=True,
            dataloader_num_workers=8,
            data_seed=SEED,
            eval_strategy="steps",
            eval_steps=100,
            save_steps=500,
            logging_dir=OUTPUT_DIR + "logging",
            logging_steps=25,
            fp16=True,
            learning_rate=2e-4,
            save_total_limit=2,
            use_cpu=False,
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=dataset["train"],
            eval_dataset=dataset["test"],
            compute_metrics=compute_metrics,
        )

        results = trainer.train()

        evaluation = evaluate_model(model, dataset, device)
        evaluation_results.append(evaluation)

        training_results.append(dict(
            model = model,
            dataset = dataset,
            training = results,
            evaluation = evaluation,
        ))

    return training_results, mean_evaluation(evaluation_results)

In [9]:
formal_register  = cross_validate('formal_register')
thematic_coherence = cross_validate('thematic_coherence')
text_typology = cross_validate('text_typology')
cohesion = cross_validate('cohesion')
results = pd.DataFrame([formal_register[1], thematic_coherence[1], text_typology[1], cohesion[1]], index=['Registro formal', 'Coerência Temática', 'Tipologia Textual', 'Coesão'])

Filter:   0%|          | 0/1188 [00:00<?, ? examples/s]

Map:   0%|          | 0/1188 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1188 [00:00<?, ? examples/s]

/home/gabriel/miniconda3/envs/project-hg/lib/python3.11/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss,Validation Loss,Accuracy,F1
100,0.707600,1.098210,0.445378,0.469912
200,0.483400,1.039030,0.449580,0.464979
300,0.375400,0.775935,0.516807,0.543003
400,0.324000,0.751126,0.552521,0.578631
500,0.270900,0.843985,0.559664,0.584952
600,0.229800,0.820156,0.568627,0.592901
700,0.213800,0.814309,0.581032,0.603483
800,0.201700,0.838298,0.590336,0.611156
900,0.203200,0.856526,0.595705,0.615256
1000,0.160300,0.825372,0.601261,0.618850


...

Step,Training Loss,Validation Loss,Accuracy,F1
100,0.751800,1.071316,0.577173,0.581640
200,0.534400,1.066015,0.576834,0.581376
300,0.391100,0.827872,0.577192,0.581727
400,0.340500,0.933733,0.577317,0.581872
500,0.259400,0.989168,0.577441,0.582031
600,0.252600,0.926635,0.577592,0.582184
700,0.225800,0.914421,0.577849,0.582446
800,0.205100,0.993802,0.577997,0.582610
900,0.185700,1.025296,0.578117,0.582752
1000,0.191400,0.989569,0.578303,0.582934


Filter:   0%|          | 0/1188 [00:00<?, ? examples/s]

Map:   0%|          | 0/1188 [00:00<?, ? examples/s]

/home/gabriel/miniconda3/envs/project-hg/lib/python3.11/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss,Validation Loss,Accuracy,F1
100,0.771900,1.290544,0.579338,0.584014
200,0.529500,0.985374,0.579257,0.583975
300,0.395600,0.977442,0.579151,0.583903
400,0.347200,0.948297,0.579249,0.584020
500,0.269600,0.919457,0.579373,0.584145
600,0.254900,1.052823,0.579306,0.584104
700,0.193800,0.971927,0.579517,0.584311
800,0.177800,1.031260,0.579638,0.584440
900,0.158200,1.076595,0.579747,0.584551
1000,0.162600,1.047576,0.579992,0.584780


In [10]:
results

,accuracy,weighted_precision,weighted_recall,weighted_f1,macro_precision,macro_recall,macro_f1
Registro formal,0.643981,0.642576,0.643981,0.641287,0.424591,0.396777,0.404402
Coerência Temática,0.624575,0.622286,0.624575,0.620606,0.399331,0.414174,0.398246
Tipologia Textual,0.521856,0.543915,0.521856,0.529059,0.368106,0.360199,0.355740
Coesão,0.650679,0.646430,0.650679,0.646595,0.430058,0.395604,0.404842
